In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import TimeSeriesSplit
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer

from datetime import datetime, timedelta
from sklearn.model_selection import KFold
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb
from scipy.stats import gmean


In [15]:
train_df =  pd.read_csv('sub_train.csv', index_col=['startdate'], parse_dates=['startdate'])
test_df  =  pd.read_csv('sub_test.csv', index_col=['startdate'], parse_dates=['startdate'])
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

climateregions__climateregion


In [44]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [55]:
test_df[target[0]] = np.nan

In [61]:
df = pd.concat([train_df[main_attrs + target], test_df[main_attrs + target]])

In [67]:
region_df = df.groupby(['lat', 'lon', 'climateregions__climateregion'])[target[0]].count().reset_index()
region_df['region_id'] = range(len(region_df))
region_df

,lat,lon,climateregions__climateregion,contest-tmp2m-14d__tmp2m,region_id
0,0.000000,0.833333,0,731,0
1,0.000000,0.866667,4,731,1
2,0.045455,0.833333,4,731,2
3,0.045455,0.866667,4,731,3
4,0.045455,0.900000,4,731,4
...,...,...,...,...,...
509,1.000000,0.733333,9,731,509
510,1.000000,0.766667,9,731,510
511,1.000000,0.800000,9,731,511
512,1.000000,0.833333,9,731,512


In [71]:
_df = df.merge(region_df[['lat', 'lon', 'climateregions__climateregion', 'region_id']], how='left', on=['lat', 'lon', 'climateregions__climateregion'])

(407088, 61)

In [72]:
train_df = _df[:len(train_df)]
test_df = _df[len(train_df):]

In [82]:
X = train_df[main_attrs + ["region_id"]]
Xtest = test_df[main_attrs + ["region_id"]]
y = train_df[target[0]]

In [80]:
from category_encoders import MEstimateEncoder
encoder = MEstimateEncoder(cols=["region_id"], m=5.0)
encoder.fit(X, y)

MEstimateEncoder(cols=['region_id'], m=5.0)

In [83]:
X_train = encoder.transform(X)
X_test = encoder.transform(Xtest)

In [84]:
cols = main_attrs + ["region_id"]

In [85]:
def train_by_df(X, y, num_iters, n_splits):
    skf = KFold(n_splits=n_splits, random_state=2048, shuffle=True)
    skf.get_n_splits(X, y)
    
    train_index_list = []
    test_index_list = []
    train_x_list = []
    val_x_list = []
    train_y_list = []
    val_y_list = []

    for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
        train_index_list.append(train_index)
        test_index_list.append(test_index)

        train_x_list.append(X[train_index, :])
        val_x_list.append(X[test_index, :])
        train_y_list.append(y[train_index])
        val_y_list.append(y[test_index])    
    cat_models = []
    for i in tqdm(range(n_splits)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=num_iters, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        cat_models.append(clf)
    return cat_models

In [86]:
def pred_by_df(models, x_test_input):
    preds = []
    for model in tqdm(models):
        preds.append(model.predict(x_test_input))
    s = 0
    for pred in preds:
        s += pred
    s = s/5
    return s

In [88]:
cat_models = train_by_df(X_train.values, train_df[target[0]].values, 5000, 5)

5it [00:04,  1.19it/s]
  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2536140	test: 9.2321711	best: 9.2321711 (0)	total: 5.62ms	remaining: 28.1s
1:	learn: 8.6859950	test: 8.6655199	best: 8.6655199 (1)	total: 12ms	remaining: 30.1s
2:	learn: 8.1619812	test: 8.1430204	best: 8.1430204 (2)	total: 20.9ms	remaining: 34.8s
3:	learn: 7.6716466	test: 7.6542751	best: 7.6542751 (3)	total: 24.8ms	remaining: 31s
4:	learn: 7.2203706	test: 7.2045464	best: 7.2045464 (4)	total: 28.7ms	remaining: 28.7s
5:	learn: 6.8025331	test: 6.7880729	best: 6.7880729 (5)	total: 32.6ms	remaining: 27.1s
6:	learn: 6.4211104	test: 6.4082783	best: 6.4082783 (6)	total: 36.5ms	remaining: 26s
7:	learn: 6.0493044	test: 6.0364915	best: 6.0364915 (7)	total: 40.3ms	remaining: 25.2s
8:	learn: 5.7200460	test: 5.7085586	best: 5.7085586 (8)	total: 44.1ms	remaining: 24.5s
9:	learn: 5.4163675	test: 5.4059954	best: 5.4059954 (9)	total: 48.2ms	remaining: 24s
10:	learn: 5.1195072	test: 5.1093991	best: 5.1093991 (10)	total: 57.5ms	remaining: 26.1s
11:	learn: 4.848555

129:	learn: 1.1136715	test: 1.1157059	best: 1.1157059 (129)	total: 606ms	remaining: 22.7s
130:	learn: 1.1101288	test: 1.1121763	best: 1.1121763 (130)	total: 610ms	remaining: 22.7s
131:	learn: 1.1067340	test: 1.1088439	best: 1.1088439 (131)	total: 614ms	remaining: 22.6s
132:	learn: 1.1028689	test: 1.1049174	best: 1.1049174 (132)	total: 617ms	remaining: 22.6s
133:	learn: 1.0999378	test: 1.1020087	best: 1.1020087 (133)	total: 621ms	remaining: 22.6s
134:	learn: 1.0974504	test: 1.0995848	best: 1.0995848 (134)	total: 625ms	remaining: 22.5s
135:	learn: 1.0940421	test: 1.0961962	best: 1.0961962 (135)	total: 628ms	remaining: 22.5s
136:	learn: 1.0915493	test: 1.0937333	best: 1.0937333 (136)	total: 636ms	remaining: 22.6s
137:	learn: 1.0887248	test: 1.0908961	best: 1.0908961 (137)	total: 643ms	remaining: 22.6s
138:	learn: 1.0857560	test: 1.0879744	best: 1.0879744 (138)	total: 646ms	remaining: 22.6s
139:	learn: 1.0824540	test: 1.0846325	best: 1.0846325 (139)	total: 650ms	remaining: 22.6s
140:	learn

266:	learn: 0.8527716	test: 0.8568875	best: 0.8568875 (266)	total: 1.21s	remaining: 21.4s
267:	learn: 0.8511837	test: 0.8552369	best: 0.8552369 (267)	total: 1.22s	remaining: 21.5s
268:	learn: 0.8503136	test: 0.8543585	best: 0.8543585 (268)	total: 1.22s	remaining: 21.4s
269:	learn: 0.8493059	test: 0.8533950	best: 0.8533950 (269)	total: 1.22s	remaining: 21.4s
270:	learn: 0.8478446	test: 0.8519397	best: 0.8519397 (270)	total: 1.23s	remaining: 21.4s
271:	learn: 0.8468144	test: 0.8509191	best: 0.8509191 (271)	total: 1.23s	remaining: 21.4s
272:	learn: 0.8455934	test: 0.8497045	best: 0.8497045 (272)	total: 1.23s	remaining: 21.4s
273:	learn: 0.8447126	test: 0.8489253	best: 0.8489253 (273)	total: 1.24s	remaining: 21.3s
274:	learn: 0.8433550	test: 0.8475843	best: 0.8475843 (274)	total: 1.24s	remaining: 21.3s
275:	learn: 0.8422862	test: 0.8464997	best: 0.8464997 (275)	total: 1.25s	remaining: 21.3s
276:	learn: 0.8410159	test: 0.8452835	best: 0.8452835 (276)	total: 1.25s	remaining: 21.3s
277:	learn

358:	learn: 0.7643786	test: 0.7697933	best: 0.7697933 (358)	total: 1.61s	remaining: 20.8s
359:	learn: 0.7632697	test: 0.7687427	best: 0.7687427 (359)	total: 1.62s	remaining: 20.8s
360:	learn: 0.7625825	test: 0.7680663	best: 0.7680663 (360)	total: 1.62s	remaining: 20.8s
361:	learn: 0.7617669	test: 0.7672606	best: 0.7672606 (361)	total: 1.62s	remaining: 20.8s
362:	learn: 0.7610958	test: 0.7665896	best: 0.7665896 (362)	total: 1.63s	remaining: 20.8s
363:	learn: 0.7604115	test: 0.7658686	best: 0.7658686 (363)	total: 1.63s	remaining: 20.8s
364:	learn: 0.7597064	test: 0.7651858	best: 0.7651858 (364)	total: 1.64s	remaining: 20.8s
365:	learn: 0.7589284	test: 0.7643707	best: 0.7643707 (365)	total: 1.64s	remaining: 20.8s
366:	learn: 0.7585087	test: 0.7640142	best: 0.7640142 (366)	total: 1.64s	remaining: 20.7s
367:	learn: 0.7577885	test: 0.7632735	best: 0.7632735 (367)	total: 1.65s	remaining: 20.7s
368:	learn: 0.7572575	test: 0.7627019	best: 0.7627019 (368)	total: 1.65s	remaining: 20.7s
369:	learn

...

4625:	learn: 0.2958885	test: 0.3219213	best: 0.3219213 (4625)	total: 20.5s	remaining: 1.66s
4626:	learn: 0.2958458	test: 0.3218747	best: 0.3218747 (4626)	total: 20.5s	remaining: 1.65s
4627:	learn: 0.2958001	test: 0.3218326	best: 0.3218326 (4627)	total: 20.5s	remaining: 1.65s
4628:	learn: 0.2957826	test: 0.3218162	best: 0.3218162 (4628)	total: 20.5s	remaining: 1.65s
4629:	learn: 0.2957567	test: 0.3217930	best: 0.3217930 (4629)	total: 20.6s	remaining: 1.64s
4630:	learn: 0.2957399	test: 0.3217779	best: 0.3217779 (4630)	total: 20.6s	remaining: 1.64s
4631:	learn: 0.2957141	test: 0.3217516	best: 0.3217516 (4631)	total: 20.6s	remaining: 1.63s
4632:	learn: 0.2956850	test: 0.3217326	best: 0.3217326 (4632)	total: 20.6s	remaining: 1.63s
4633:	learn: 0.2956542	test: 0.3217046	best: 0.3217046 (4633)	total: 20.6s	remaining: 1.62s
4634:	learn: 0.2956169	test: 0.3216763	best: 0.3216763 (4634)	total: 20.6s	remaining: 1.62s
4635:	learn: 0.2955851	test: 0.3216516	best: 0.3216516 (4635)	total: 20.6s	remai

4719:	learn: 0.2933946	test: 0.3196074	best: 0.3196074 (4719)	total: 20.9s	remaining: 1.24s
4720:	learn: 0.2933757	test: 0.3195866	best: 0.3195866 (4720)	total: 20.9s	remaining: 1.24s
4721:	learn: 0.2933577	test: 0.3195729	best: 0.3195729 (4721)	total: 20.9s	remaining: 1.23s
4722:	learn: 0.2933340	test: 0.3195430	best: 0.3195430 (4722)	total: 20.9s	remaining: 1.23s
4723:	learn: 0.2933045	test: 0.3195186	best: 0.3195186 (4723)	total: 21s	remaining: 1.22s
4724:	learn: 0.2932667	test: 0.3194837	best: 0.3194837 (4724)	total: 21s	remaining: 1.22s
4725:	learn: 0.2932380	test: 0.3194596	best: 0.3194596 (4725)	total: 21s	remaining: 1.22s
4726:	learn: 0.2931761	test: 0.3193978	best: 0.3193978 (4726)	total: 21s	remaining: 1.21s
4727:	learn: 0.2931383	test: 0.3193673	best: 0.3193673 (4727)	total: 21s	remaining: 1.21s
4728:	learn: 0.2931069	test: 0.3193308	best: 0.3193308 (4728)	total: 21s	remaining: 1.2s
4729:	learn: 0.2930818	test: 0.3193120	best: 0.3193120 (4729)	total: 21s	remaining: 1.2s
4730

4846:	learn: 0.2899022	test: 0.3163866	best: 0.3163866 (4846)	total: 21.5s	remaining: 679ms
4847:	learn: 0.2898762	test: 0.3163582	best: 0.3163582 (4847)	total: 21.5s	remaining: 675ms
4848:	learn: 0.2898534	test: 0.3163259	best: 0.3163259 (4848)	total: 21.5s	remaining: 671ms
4849:	learn: 0.2898329	test: 0.3163064	best: 0.3163064 (4849)	total: 21.5s	remaining: 666ms
4850:	learn: 0.2898112	test: 0.3162875	best: 0.3162875 (4850)	total: 21.5s	remaining: 662ms
4851:	learn: 0.2897908	test: 0.3162727	best: 0.3162727 (4851)	total: 21.5s	remaining: 657ms
4852:	learn: 0.2897592	test: 0.3162460	best: 0.3162460 (4852)	total: 21.5s	remaining: 653ms
4853:	learn: 0.2897344	test: 0.3162231	best: 0.3162231 (4853)	total: 21.6s	remaining: 648ms
4854:	learn: 0.2897181	test: 0.3162078	best: 0.3162078 (4854)	total: 21.6s	remaining: 644ms
4855:	learn: 0.2896868	test: 0.3161728	best: 0.3161728 (4855)	total: 21.6s	remaining: 639ms
4856:	learn: 0.2896625	test: 0.3161515	best: 0.3161515 (4856)	total: 21.6s	remai

4966:	learn: 0.2868660	test: 0.3135703	best: 0.3135703 (4966)	total: 22.1s	remaining: 147ms
4967:	learn: 0.2868399	test: 0.3135446	best: 0.3135446 (4967)	total: 22.1s	remaining: 143ms
4968:	learn: 0.2868176	test: 0.3135189	best: 0.3135189 (4968)	total: 22.1s	remaining: 138ms
4969:	learn: 0.2867813	test: 0.3134804	best: 0.3134804 (4969)	total: 22.1s	remaining: 134ms
4970:	learn: 0.2867618	test: 0.3134582	best: 0.3134582 (4970)	total: 22.2s	remaining: 129ms
4971:	learn: 0.2867380	test: 0.3134320	best: 0.3134320 (4971)	total: 22.2s	remaining: 125ms
4972:	learn: 0.2867092	test: 0.3134056	best: 0.3134056 (4972)	total: 22.2s	remaining: 120ms
4973:	learn: 0.2866727	test: 0.3133587	best: 0.3133587 (4973)	total: 22.2s	remaining: 116ms
4974:	learn: 0.2866476	test: 0.3133381	best: 0.3133381 (4974)	total: 22.2s	remaining: 111ms
4975:	learn: 0.2866271	test: 0.3133155	best: 0.3133155 (4975)	total: 22.2s	remaining: 107ms
4976:	learn: 0.2865882	test: 0.3132800	best: 0.3132800 (4976)	total: 22.2s	remai

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:27<00:00, 29.41s/it]


In [91]:
X_test

,contest-wind-h500-14d__wind-hgt-500,contest-slp-14d__slp,nmme-tmp2m-34w__ccsm3,elevation__elevation,lon,contest-prwtr-eatm-14d__prwtr,lat,climateregions__climateregion,contest-pres-sfc-gauss-14d__pres,season_sin,day_of_year_sin,contest-precip-14d__precip,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-prate-34w__cfsv2,nmme-prate-34w__nasa,nmme-prate-56w__gfdlflora,wind-uwnd-250-2010-7,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-34w__nmmemean,nmme0-prate-34w__ccsm30,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,nmme0-prate-56w__cfsv20,nmme-prate-34w__cancm3,contest-rhum-sig995-14d__rhum,nmme-prate-34w__gfdlflorb,wind-hgt-850-2010-4,contest-wind-vwnd-250-14d__wind-vwnd-250,wind-hgt-100-2010-2,wind-uwnd-250-2010-18,wind-hgt-10-2010-5,wind-uwnd-250-2010-15,wind-uwnd-250-2010-4,nmme0-prate-56w__nasa0,nmme0-prate-34w__cfsv20,wind-vwnd-250-2010-10,contest-wind-h10-14d__wind-hgt-10,wind-uwnd-925-2010-15,wind-vwnd-250-2010-3,nmme-prate-34w__cancm4,sst-2010-4,nmme0-prate-56w__ccsm30,wind-uwnd-250-2010-16,nmme0-prate-34w__gfdl0,nmme0-prate-56w__cancm40,sst-2010-1,sst-2010-3,wind-uwnd-250-2010-14,nmme0-prate-34w__gfdlflora0,nmme-prate-34w__gfdl,wind-hgt-850-2010-9,wind-vwnd-250-2010-1,sst-2010-5,cancm30,nmme-prate-34w__ccsm4,nmme0-prate-34w__nasa0,wind-hgt-500-2010-9,nmme0-prate-34w__cancm30,wind-vwnd-250-2010-13,region_id
375734,5916.86,101441.09,27.37,200.0,0.833333,40.29,0.0,0,98734.98,-1.0,-8.587640e-01,34.85,1.14,32.67,28.50,30.67,-40.56,3.12,25.09,12.24,1549.82,-4.38,29.50,20.01,72.04,33.57,1023.51,-4.82,-6044.99,142.32,7408.07,-75.49,221.61,19.67,40.72,72.60,31233.02,57.53,-210.45,21.62,41.38,16.12,-13.39,8.88,11.48,345.33,-19.05,2.89,19.03,15.33,-337.74,-176.44,12.53,29.44,29.60,24.46,-529.25,15.81,57.01,23.395958
375735,5917.85,101449.71,27.37,200.0,0.833333,40.15,0.0,0,98745.78,-1.0,-8.498171e-01,27.46,0.55,32.67,28.50,30.67,-41.01,3.22,25.09,12.24,1550.55,-4.50,29.50,20.01,71.93,33.57,815.95,-4.34,-6152.43,140.27,6913.60,-70.65,221.58,19.67,40.72,67.94,31230.63,45.23,-211.43,21.62,39.57,16.12,-8.10,8.88,11.48,344.50,-19.20,-7.43,19.03,15.33,-243.39,-158.62,12.27,29.44,29.60,24.46,-438.92,15.81,56.24,23.395958
375736,5918.56,101446.14,27.37,200.0,0.833333,40.07,0.0,0,98743.07,-1.0,-8.406184e-01,24.65,0.12,32.67,28.50,30.67,-51.24,3.42,25.09,12.24,1550.16,-4.61,29.50,20.01,72.43,33.57,619.22,-3.62,-6296.21,137.79,6445.81,-66.89,221.06,19.67,40.72,66.51,31225.26,33.94,-209.54,21.62,37.79,16.12,-11.03,8.88,11.48,343.40,-19.35,-12.71,19.03,15.33,-51.71,-138.70,11.94,29.44,29.60,24.46,-325.51,15.81,45.71,23.395958
375737,5920.30,101443.27,27.37,200.0,0.833333,40.00,0.0,0,98748.59,-1.0,-8.311706e-01,22.43,0.26,32.67,28.50,30.67,-61.72,3.82,25.09,12.24,1550.15,-4.49,29.50,20.01,71.88,33.57,459.28,-2.80,-6417.62,134.52,5895.94,-73.97,218.18,19.67,40.72,75.15,31220.87,29.31,-202.11,21.62,35.93,16.12,-23.55,8.88,11.48,341.91,-19.38,-17.76,19.03,15.33,191.32,-122.68,11.34,29.44,29.60,24.46,-274.45,15.81,41.89,23.395958
375738,5923.03,101448.09,27.37,200.0,0.833333,39.80,0.0,0,98761.84,-1.0,-8.214766e-01,16.31,0.65,32.67,28.50,30.67,-66.26,3.91,25.09,12.24,1551.19,-4.34,29.50,20.01,71.18,33.57,326.11,-2.59,-6479.77,126.65,5383.52,-81.71,214.43,19.67,40.72,91.43,31214.54,30.52,-187.25,21.62,33.97,16.12,-26.73,8.88,11.48,340.22,-19.27,-18.64,19.03,15.33,290.21,-113.51,10.56,29.44,29.60,24.46,-279.28,15.81,68.04,23.395958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407083,5675.63,101614.98,-0.65,100.0,0.866667,14.63,1.0,9,97766.83,0.0,-6.880243e-02,10.28,28.72,25.75,18.07,15.88,11.55,2.31,17.85,19.90,1482.82,5.42,39.38,15.75,84.04,13.17,83.41,-8.44,-2239.27,-103.19,8744.09,55.53,-289.95,49.26,32.20,-1.09,30269.05,4.48,149.53,15.00,-39.74,10.43,1.38,20.26,22.37,112.99,9.68,200.48,11.33,17.52,1945.79,79.36,17.62

In [92]:
s = pred_by_df(cat_models, X_test)
df = pd.read_csv('./sample_solution.csv')
best_df = pd.read_csv('./best_sub.csv')
y_best = best_df[target[0]].values
df['contest-tmp2m-14d__tmp2m'] = s * 0.1 + y_best * 0.9
df.to_csv('submission_target_encoding.csv', index=False)

from numpy import dot
from numpy.linalg import norm
cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 37.93it/s]


cos_sim with best submission: 0.9997494166500239
